# Ocena czasu potrzebnego na dekodowanie filmu

Na podstawie zbioru (https://archive.ics.uci.edu/dataset/335/online+video+characteristics+and+transcoding+time+dataset)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Opis danych wejściowych i ich analiza

Rzut oka na pierwsze pięć wierszy dwóch podanych zbiorów danych za pomocą funkcji head(). Dla pierwszego zbioru danych (data1) dostępne są 22 atrybuty podane poniżej. Drugi zbiór danych (data2) reprezentuje 11 atrybutów.

In [3]:
data1 = pd.read_csv('transcoding_mesurment.tsv', sep='\t')
data1.head()

,id,duration,codec,width,height,bitrate,framerate,i,p,b,...,p_size,b_size,size,o_codec,o_bitrate,o_framerate,o_width,o_height,umem,utime
0,04t6-jw9czg,130.35667,mpeg4,176,144,54590,12.0,27,1537,0,...,825054,0,889537,mpeg4,56000,12.0,176,144,22508,0.612
1,04t6-jw9czg,130.35667,mpeg4,176,144,54590,12.0,27,1537,0,...,825054,0,889537,mpeg4,56000,12.0,320,240,25164,0.980
2,04t6-jw9czg,130.35667,mpeg4,176,144,54590,12.0,27,1537,0,...,825054,0,889537,mpeg4,56000,12.0,480,360,29228,1.216
3,04t6-jw9czg,130.35667,mpeg4,176,144,54590,12.0,27,1537,0,...,825054,0,889537,mpeg4,56000,12.0,640,480,34316,1.692
4,04t6-jw9czg,130.35667,mpeg4,176,144,54590,12.0,27,1537,0,...,825054,0,889537,mpeg4,56000,12.0,1280,720,58528,3.456


In [4]:
data2 = pd.read_csv('youtube_videos.tsv', sep='\t')
data2.head()

,id,duration,bitrate,bitrate(video),height,width,frame rate,frame rate(est.),codec,category,url
0,uDNj-_5ty48,267,373,274,568,320,29.97,0.00,h264,Music,http://r2---sn-ovgq0oxu-5goe.c.youtube.com/vid...
1,uDNj-_5ty48,267,512,396,480,270,29.97,29.97,h264,Music,http://r2---sn-ovgq0oxu-5goe.c.youtube.com/vid...
2,uDNj-_5ty48,267,324,263,400,226,29.97,29.97,flv1,Music,http://r2---sn-ovgq0oxu-5goe.c.youtube.com/vid...
3,uDNj-_5ty48,267,85,55,176,144,12.00,12.00,mpeg4,Music,http://r2---sn-ovgq0oxu-5goe.c.youtube.com/vid...
4,WCgt-AactyY,31,1261,1183,640,480,24.00,0.00,h264,People & Blogs,http://r1---sn-ovgq0oxu-5goe.c.youtube.com/vid...


Zapoznanie się z krótkim opisem danych za pomocą funkcji info().

In [5]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68784 entries, 0 to 68783
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           68784 non-null  object 
 1   duration     68784 non-null  float64
 2   codec        68784 non-null  object 
 3   width        68784 non-null  int64  
 4   height       68784 non-null  int64  
 5   bitrate      68784 non-null  int64  
 6   framerate    68784 non-null  float64
 7   i            68784 non-null  int64  
 8   p            68784 non-null  int64  
 9   b            68784 non-null  int64  
 10  frames       68784 non-null  int64  
 11  i_size       68784 non-null  int64  
 12  p_size       68784 non-null  int64  
 13  b_size       68784 non-null  int64  
 14  size         68784 non-null  int64  
 15  o_codec      68784 non-null  object 
 16  o_bitrate    68784 non-null  int64  
 17  o_framerate  68784 non-null  float64
 18  o_width      68784 non-null  int64  
 19  o_he

Zbiór pierwszy składa się z 68784 przykładów i wydaje się niezbyt dużym zbiorem na potrzeby uczenia maszynowego, jednak jest idealny do demonstracji jego działania. Zauważyć można, że większość atrybutów zbioru ma wartości numeryczne (float64 i int64). 

Wyjątek stanowią trzy pola: id, codec i o_codec, których typem wartości jest object. Object jest nadrzędną klasą wszystkich klas w języku Python, co oznacza, że pola te mogą przechowywać dowolny obiekt stworzony w Pythonie (czyli dla pliku TSV każdy obiekt tekstowy). Po przyjrzeniu się wynikowi funkcji data1.head() nietrudno zauważyć, że wyniki tych trzech pól są powtarzalne, z czego wywnioskować można, że stanowią one wartości wyliczeniowe enum. 

Warto więc sprawdzić te wartości dodatkowo zliczając częstotliwość ich występowania funkcją value_counts().

In [6]:
data1["id"].value_counts()

id
04t6-jw9czg    841
2ApZ-KZ-pEk    841
3fpg-LlHcks    841
3fmy-4uvR9U    841
3_fF9rtYCQ0    841
              ... 
GXn0-yo-L0I      1
gZDG-8Te3tg      1
h52Y-SSyvoM      1
H5ae-PJpkwI      1
zZKo-QsY86U      1
Name: count, Length: 1099, dtype: int64

In [7]:
data1["codec"].value_counts()

codec
h264     31545
vp8      18387
mpeg4    12012
flv       6840
Name: count, dtype: int64

In [8]:
data1["o_codec"].value_counts()

o_codec
mpeg4    17291
vp8      17277
flv      17135
h264     17081
Name: count, dtype: int64

Id ma więc zarówno te wartości, które się powtarzają, jak i te, które w zbiorze są unikatowe. Pole codec ma 4 różne wartości, które może przybierać i żadna z nich nie występuje w zbiorze tylko jeden raz. Tak samo jest z wartościami o_codec. Dodatkowo zauważyć można, że wartości w kolumnie codec i o_codec są takie same, tylko ich częstotliwość występowania jest różna.

## Analiza eksploracyjna

Analiza reszty pól metodą describe(), która daje możliwość podsumowania atrybutów numerycznych.

In [9]:
data1.describe()

,duration,width,height,bitrate,framerate,i,p,b,frames,i_size,p_size,b_size,size,o_bitrate,o_framerate,o_width,o_height,umem,utime
count,68784.000000,68784.000000,68784.000000,6.878400e+04,68784.000000,68784.000000,68784.000000,68784.000000,68784.000000,6.878400e+04,6.878400e+04,68784.0,6.878400e+04,6.878400e+04,68784.000000,68784.000000,68784.000000,68784.000000,68784.000000
mean,286.413921,624.934171,412.572226,6.937015e+05,23.241321,100.868312,6531.692210,9.147854,6641.708377,2.838987e+06,2.218057e+07,0.0,2.502294e+07,1.395036e+06,21.190862,802.336357,503.825541,228224.717900,9.996355
std,287.257650,463.169069,240.615472,1.095628e+06,7.224848,84.764791,6075.871744,92.516177,6153.342453,4.325137e+06,5.097306e+07,0.0,5.414402e+07,1.749352e+06,6.668703,609.959797,315.970438,97430.878373,16.107429
min,31.080000,176.000000,144.000000,8.384000e+03,5.705752,7.000000,175.000000,0.000000,192.000000,1.164800e+04,3.384500e+04,0.0,1.918790e+05,5.600000e+04,12.000000,176.000000,144.000000,22508.000000,0.184000
25%,106.765000,320.000000,240.000000,1.343340e+05,15.000000,39.000000,2374.000000,0.000000,2417.000000,3.933950e+05,1.851539e+06,0.0,2.258222e+06,1.090000e+05,15.000000,320.000000,240.000000,216820.000000,2.096000
50%,239.141660,480.000000,360.000000,2.911500e+05,25.021740,80.000000,5515.000000,0.000000,5628.000000,9.458650e+05,6.166260e+06,0.0,7.881069e+06,5.390000e+05,24.000000,480.000000,360.000000,219480.000000,4.408000
75%,379.320000,640.000000,480.000000,6.529670e+05,29.000000,138.000000,9155.000000,0.000000,9232.000000,3.392479e+06,1.515506e+07,0.0,1.977335e+07,3.000000e+06,25.000000,1280.000000,720.000000,219656.000000,10.433000
max,25844.086000,1920.000000,1080.000000,7.628466e+06,48.000000,5170.000000,304959.000000,9407.000000,310129.000000,9.082855e+07,7.689970e+08,0.0,8.067111e+08,5.000000e+06,29.970000,1920.000000,1080.000000,711824.000000,224.574000


Z wywołania funkcji describe() wynika, że dla każdej kolumny istnieją 68784 niepuste wartości, co oznacza, że w zbiorze danych nie ma wartości pustych dla żadnej kolumny.

